# Quick start with OceanBase vector search using SQL

This notebook demonstrates how to implement vector search using OceanBase Database. You will learn how to create vector tables, insert vector data, and perform similarity searches using SQL.

OceanBase supports efficient vector search directly with SQL. OceanBase Database's vector search is built for multi-modal integration, offering unified queries, scalability, high performance, high availability, low cost, multi-tenancy, and data security. For more details, see the [Overview of vector search](https://en.oceanbase.com/docs/common-oceanbase-database-10000000001976351).

> **Note:** This tutorial uses MySQL-compatible mode as an example.

## How to use this notebook

### Option 1: Jupyter Notebook / JupyterLab
1. Install Jupyter: `pip install jupyter` or `pip install jupyterlab`
2. Open terminal and navigate to the notebook directory
3. Run: `jupyter notebook` or `jupyter lab`
4. Open this `.ipynb` file from the interface

### Option 2: VS Code
1. Install the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) for VS Code
2. Open this `.ipynb` file in VS Code
3. Select a Python kernel when prompted

### Option 3: Google Colab
1. Go to [Google Colab](https://colab.research.google.com/)
2. Click "File" → "Upload notebook"
3. Upload this `.ipynb` file

### Running the notebook
- **Run all cells**: Click "Run" → "Run All Cells" (or press `Shift + Enter` on each cell)
- **Run one cell**: Click on a cell and press `Shift + Enter`
- **Expected time**: The entire notebook takes about 1-2 minutes to complete

> **💡 Tip:** The database connection is pre-configured, so you can run all cells directly without any modifications!

## Prerequisites

Before you begin, ensure you have:

- Python 3.7 or higher installed
- Network access to the OceanBase Database instance

> **Note:** The database connection is pre-configured in this notebook. You can run the cells directly without any additional setup. If you prefer to use your own database, you can modify the connection settings in the configuration cell.

## Install Python requirements

Before you start, you need to install all required Python dependencies.

In [1]:
!pip install -q pymysql


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pymysql
import sys

# Database connection configuration
# The connection settings below are pre-configured and ready to use.
# If you need to use your own database, you can modify these values.
host = "obmt7bftsnwuc9z4-mi.aliyun-cn-hangzhou-internet.oceanbase.cloud"
port = 3306
user = "jackietest3"
password = "Nl]03?yN"
database = "jackic-test-3"

# Table name (used throughout the notebook)
TABLE_NAME = "t1"

## Connect to OceanBase Database

The database connection is pre-configured in the previous cell. Run the cell below to connect to the database.

> **Note:** If you prefer to use the command line, you can connect using: `obclient -h <host> -P <port> -u <user> -p`

In [3]:
# Connect to the database
try:
    conn = pymysql.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database,
        charset="utf8mb4"
    )
    cursor = conn.cursor()
    print("✅ Database connection successful!")
    print(f"   Host: {host}")
    print(f"   Port: {port}")
    print(f"   User: {user}")
    print(f"   Database: {database}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    print("\nTip: Please check your network connectivity and ensure the database is accessible")
    conn = None
    cursor = None

✅ Database connection successful!
   Host: obmt7bftsnwuc9z4-mi.aliyun-cn-hangzhou-internet.oceanbase.cloud
   Port: 3306
   User: jackietest3
   Database: jackic-test-3


## Create index and mappings for vector table

Before you can index documents into OceanBase, you need to create a table with the correct schema and vector index configuration.

When creating a table, use the `VECTOR(dim)` data type to declare a vector column and specify its dimension. You will also need to create a vector index on this column, specifying at least the `type` and `distance` parameters.

This example creates a vector column called `embedding` with dimension `3`, and adds an HNSW index using `L2` distance.

In [4]:
# Check if database connection exists
if 'conn' not in globals() or conn is None:
    print("❌ Error: Database connection not established.")
    print("Please run the 'Connect to OceanBase Database' cell first.")
    raise ConnectionError("Database connection required. Please run the connection cell first.")

# Ensure TABLE_NAME is defined (defined in configuration cell, but check here for safety)
if 'TABLE_NAME' not in globals():
    TABLE_NAME = "t1"

# Flag to check if table should be deleted before creating
SHOULD_DELETE_TABLE = True

# Table schema definition
CREATE_TABLE_SQL = """
CREATE TABLE t1( 
    id INT PRIMARY KEY, 
    doc VARCHAR(200), 
    embedding VECTOR(3), 
    VECTOR INDEX idx1(embedding) WITH (distance=L2, type=hnsw) 
)
"""

# Check if we want to delete table before creating
if SHOULD_DELETE_TABLE:
    try:
        cursor.execute(f"USE `{database}`")
        cursor.execute(f"DROP TABLE IF EXISTS {TABLE_NAME}")
        conn.commit()
        print(f"ℹ️  Deleted existing table {TABLE_NAME} (if exists)")
    except Exception as e:
        print(f"ℹ️  No existing table to delete: {e}")

# Create the vector table
try:
    cursor.execute(f"USE `{database}`")
    cursor.execute(CREATE_TABLE_SQL)
    conn.commit()
    print(f"✅ Created table {TABLE_NAME}")
    print("   Table schema:")
    print("   - id: INT PRIMARY KEY")
    print("   - doc: VARCHAR(200)")
    print("   - embedding: VECTOR(3)")
    print("   - Vector index: idx1 (L2 distance, HNSW type)")
except Exception as e:
    print(f"❌ Failed to create table: {e}")
    print("\nTip: Please check database connection and user permissions (requires CREATE TABLE permission)")
    raise

ℹ️  Deleted existing table t1 (if exists)
✅ Created table t1
   Table schema:
   - id: INT PRIMARY KEY
   - doc: VARCHAR(200)
   - embedding: VECTOR(3)
   - Vector index: idx1 (L2 distance, HNSW type)


> **Note:** For large datasets, it is recommended to import data first and then create the vector index. For details about creating indexes after data import, see [Create a vector index](https://en.oceanbase.com/docs/common-oceanbase-database-10000000002440117).

## Insert vector data

To simulate a vector search scenario, we need to construct some vector data first. Each row of data includes a description and its corresponding vector. 

In this example, we use a simple dataset with 6 items:
- **Fruits**: Apple, Banana, Orange (with similar vector values around [0.6-1.2])
- **Vegetables**: Carrot, Spinach, Tomato (with similar vector values around [4.8-5.4])

This example assumes that `'Apple'` corresponds to the vector `'[1.2,0.7,1.1]'`, and `'Carrot'` corresponds to the vector `'[5.3,4.8,5.4]'`, and so on.

In [5]:
# Check if database connection exists
if 'conn' not in globals() or conn is None or 'cursor' not in globals() or cursor is None:
    print("❌ Error: Database connection not established.")
    print("Please run the 'Connect to OceanBase Database' cell first.")
    raise ConnectionError("Database connection required. Please run the connection cell first.")

# Sample vector data
# Each row contains: (id, description, vector_embedding)
sample_data = [
    (1, 'Apple', '[1.2,0.7,1.1]'),
    (2, 'Banana', '[0.6,1.2,0.8]'),
    (3, 'Orange', '[1.1,1.1,0.9]'),
    (4, 'Carrot', '[5.3,4.8,5.4]'),
    (5, 'Spinach', '[4.9,5.3,4.8]'),
    (6, 'Tomato', '[5.2,4.9,5.1]')
]

# Clear old data from the table (if exists)
try:
    cursor.execute(f"USE `{database}`")
    cursor.execute(f"DELETE FROM {TABLE_NAME}")
    conn.commit()
    print(f"ℹ️  Cleared existing data from table {TABLE_NAME}")
except Exception as e:
    print(f"ℹ️  No existing data to clear: {e}")

# Insert vector data
try:
    cursor.execute(f"USE `{database}`")
    
    insert_sql = f"""
    INSERT INTO {TABLE_NAME} VALUES 
        (%s, %s, %s)
    """
    
    cursor.executemany(insert_sql, sample_data)
    conn.commit()
    
    # Verify insertion result
    cursor.execute(f"SELECT COUNT(*) FROM {TABLE_NAME}")
    count = cursor.fetchone()[0]
    
    print(f"✅ Inserted {count} documents")
    print("   Sample data:")
    print("   - Fruits: Apple, Banana, Orange")
    print("   - Vegetables: Carrot, Spinach, Tomato")
    
except Exception as e:
    print(f"❌ Failed to insert data: {e}")
    print("\nTip: Please ensure the table has been created and check user permissions (requires INSERT permission)")
    raise

ℹ️  Cleared existing data from table t1
✅ Inserted 6 documents
   Sample data:
   - Fruits: Apple, Banana, Orange
   - Vegetables: Carrot, Spinach, Tomato


> **Note:** For demonstration purposes, this example simplifies the vector dimensions to only 3 dimensions, and the vectors are manually generated. In real-world applications, you need to use embedding models to generate vectors from actual text, and the dimensions can reach hundreds or thousands.

### Verify inserted data

You can query the table data to verify that the data was inserted successfully.

In [6]:
# Query table data
try:
    if 'cursor' not in globals() or cursor is None:
        raise ConnectionError("Database connection required. Please run the connection cell first.")
    
    cursor.execute(f"USE `{database}`")
    cursor.execute(f"SELECT * FROM {TABLE_NAME}")
    results = cursor.fetchall()
    
    print("Query results:")
    print("+" + "-" * 4 + "+" + "-" * 12 + "+" + "-" * 20 + "+")
    print(f"| {'id':<4} | {'doc':<12} | {'embedding':<20} |")
    print("+" + "-" * 4 + "+" + "-" * 12 + "+" + "-" * 20 + "+")
    for row in results:
        embedding_str = str(row[2])[:18] + "..." if len(str(row[2])) > 18 else str(row[2])
        print(f"| {row[0]:<4} | {row[1]:<12} | {embedding_str:<20} |")
    print("+" + "-" * 4 + "+" + "-" * 12 + "+" + "-" * 20 + "+")
    print(f"{len(results)} rows in set")
    
except Exception as e:
    print(f"❌ Failed to query data: {e}")
    raise

Query results:
+----+------------+--------------------+
| id   | doc          | embedding            |
+----+------------+--------------------+
| 1    | Apple        | [1.2,0.7,1.1]        |
| 2    | Banana       | [0.6,1.2,0.8]        |
| 3    | Orange       | [1.1,1.1,0.9]        |
| 4    | Carrot       | [5.3,4.8,5.4]        |
| 5    | Spinach      | [4.9,5.3,4.8]        |
| 6    | Tomato       | [5.2,4.9,5.1]        |
+----+------------+--------------------+
6 rows in set


## Query the vector dataset

The next step is to run a query to search for similar vectors. The example query searches for items similar to the query vector `[0.9, 1.0, 0.9]`, which represents the concept of "fruits" in our simplified vector space.

The process is carried out with a single SQL query using the `l2_distance` function with the `APPROXIMATE` keyword for efficient approximate nearest neighbor search.

```sql
SELECT id, doc 
FROM t1 
ORDER BY l2_distance(embedding, '[0.9, 1.0, 0.9]') APPROXIMATE LIMIT 3;
```

In [ ]:
# Check if database connection exists
if 'cursor' not in globals() or cursor is None:
    print("❌ Error: Database connection not established.")
    print("Please run the 'Connect to OceanBase Database' cell first.")
    raise ConnectionError("Database connection required. Please run the connection cell first.")

# Search query
# The query vector represents "fruits" in our simplified vector space
QUERY_VECTOR = '[0.9, 1.0, 0.9]'
LIMIT = 3

# Vector search query
query = {
    "query_vector": QUERY_VECTOR,
    "limit": LIMIT
}

try:
    cursor.execute(f"USE `{database}`")
    
    search_sql = f"""
    SELECT id, doc 
    FROM {TABLE_NAME} 
    ORDER BY l2_distance(embedding, '{query['query_vector']}') APPROXIMATE LIMIT {query['limit']}
    """
    
    cursor.execute(search_sql)
    results = cursor.fetchall()
    
    print(f"Query vector: {query['query_vector']}")
    print(f"Searching for top {query['limit']} similar items...\n")
    print("Search results:")
    print("+" + "-" * 4 + "+" + "-" * 12 + "+")
    print(f"| {'id':<4} | {'doc':<12} |")
    print("+" + "-" * 4 + "+" + "-" * 12 + "+")
    for row in results:
        print(f"| {row[0]:<4} | {row[1]:<12} |")
    print("+" + "-" * 4 + "+" + "-" * 12 + "+")
    print(f"{len(results)} rows in set")
    
    print("\n✅ Search completed successfully!")
    print("   The results show the top 3 items most similar to the query vector.")
    print("   As expected, all results are fruits (Apple, Banana, Orange),")
    print("   which have vector values closer to [0.9, 1.0, 0.9] than vegetables.")
    
except Exception as e:
    print(f"❌ Vector search failed: {e}")
    raise

## Summary

You have successfully:
- ✅ Connected to OceanBase Database
- ✅ Created a vector table with HNSW index
- ✅ Inserted vector data
- ✅ Performed vector similarity search

You are now ready to use OceanBase vector search with SQL. For more advanced scenarios, refer to the official documentation or explore embedding models for generating vectors from real-world data.

### Next steps

- Explore different distance metrics (L2, cosine, inner product)
- Try different vector dimensions and index types
- Integrate with embedding models for real-world text/image search
- Scale to larger datasets with optimized indexing strategies

## Clean up resources

After completing the experiment, you can clean up the test data and close the database connection.

> **Note:** Set `CLEANUP_ENABLED = True` in the cell below to execute cleanup. Set it to `False` to skip cleanup and keep the test data.

In [12]:
# Clean up resources (optional)
# Set CLEANUP_ENABLED = True to clean up test data and close database connection
# Set CLEANUP_ENABLED = False to skip cleanup and keep the test data

CLEANUP_ENABLED = True  # Change to True to enable cleanup

try:
    if 'cursor' not in globals() or cursor is None:
        print("ℹ️  No database connection to clean up")
    elif not CLEANUP_ENABLED:
        print("ℹ️  Cleanup is disabled (CLEANUP_ENABLED = False)")
        print("   To clean up resources, set CLEANUP_ENABLED = True and run this cell again.")
    else:
        # Clean up test data
        cursor.execute(f"USE `{database}`")
        cursor.execute(f"DROP TABLE IF EXISTS {TABLE_NAME}")
        conn.commit()
        print(f"✅ Test table {TABLE_NAME} dropped")
        
        # Close database connection
        cursor.close()
        conn.close()
        print("✅ Database connection closed")
        print("\n🎉 Cleanup completed successfully!")
        
except Exception as e:
    print(f"❌ Cleanup failed: {e}")
    print("   The test data and connection remain intact.")

✅ Test table t1 dropped
✅ Database connection closed

🎉 Cleanup completed successfully!
